In [1]:
import pandas as pd
import numpy as np
import yaml
import pickle
import logging
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import country_converter as coco
from utilities.utilities import read_ac_data, read_gdp_data, read_cdd_data, saturation, availability, exposure_function, fill_missing_country_gdp_data
from utilities.plotting import plot_exposure_contour

In [2]:
# Load configuration from config.yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Set logging level
logging.basicConfig(level=config["logging_level"])

In [3]:
# Read the cooling degree days data
dd_data = read_cdd_data(config['ref_year'], config['cdd_historical_file'])

# Read the GDP data
gdp_data = read_gdp_data(str(config['ref_year']), config['gdp_historical_file'])
gdp_data = fill_missing_country_gdp_data(config['ref_year'], gdp_data, config)

# Merge the two dataframes
merged_data = pd.merge(dd_data, gdp_data, on='ISO3')

In [7]:
# Read the air conditioning data

def read_ac_data_iea(infile):
    # Add AC column to merged_data
    ac_data = pd.read_csv(infile, skiprows=3)
    # Name first column ISO3 and second column AC
    ac_data = ac_data.rename(columns={ac_data.columns[0]: "ISO3", ac_data.columns[1]: "AC"})
    # Rename country names to ISO3
    ac_data["ISO3"] = coco.convert(names=ac_data["ISO3"].tolist(), to='ISO3')
    # AC Given in percent
    ac_data["AC"] /= 100.
    return ac_data

def add_gdp_cdd_data(cdds, gdps, ac_df, year):
    """
    Add GDP and CDD data to the ac_df dataframe
    """
    # Read the cooling degree days data
    dd_data = read_cdd_data(year, config['cdd_historical_file'])
    # Read the GDP data
    gdp_data = read_gdp_data(str(year), config['gdp_historical_file'])
    # Add GDP and CDD data to the ac_df dataframe
    ac_df = pd.merge(ac_df, dd_data, on='ISO3')
    ac_df = pd.merge(ac_df, gdp_data, on='ISO3')
    return ac_df



ac_data = read_ac_data_iea(config['ac_historical_file_iea'])
ac_data = add_gdp_cdd_data(dd_data, gdp_data, ac_data, 2018)
print(ac_data)


  ISO3    AC      DD_mean           GDP
0  JPN  0.91   481.315526  42599.096879
1  USA  0.90   604.579562  62575.425728
2  KOR  0.86   651.360792  42805.208346
3  SAU  0.63  3187.489303  48668.394828
4  CHN  0.60   533.091995  15436.675531
5  MEX  0.16  1471.855206  20683.780633
6  BRA  0.16  2508.361929  14911.982952
7  IDN  0.09  2405.041162  11625.380029
8  ZAF  0.06   927.694056  14270.121191
9  IND  0.05  2615.146210   6564.876470
